In [1]:
import os
import dendropy
import pandas as pd
import numpy as np
from plumbum import local
from datetime import datetime, timedelta

symbols = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
symbol_map = {i+7: s for i, s in enumerate(symbols)}
fasttree = local["../FastTree"]
fasttree_pseudo = fasttree['-pseudo']
fasttree_pseudo_trans_unified = fasttree_pseudo['-trans', '/home/dcsoft/s/Ofir/unified_for_fastree_asaf.mat']

def transpose_dict(d):
    td = dict()
    for k1 in d:
        for k2 in d[k1]:
            td.setdefault(k2, dict())[k1] = d[k1][k2]
    return td


def fix_directories(base_dir, ind_name, delta_days=0):
    """
    make sure the dir base_path/curr_date/ind_name exists and create otherwise
    """
    curr_date = "{:%d_%m_%Y}".format(datetime.now() - timedelta(days=delta_days))
    dirs_to_create = "{}/{}/{}".format(base_dir,curr_date,ind_name)
    if not os.path.exists(dirs_to_create):
        os.makedirs(dirs_to_create)
    return dirs_to_create


def add_root_to_dict(
        textual_mutation_dict,
        cells_to_be_used_as_root,
):
    new_d = dict()
    if cells_to_be_used_as_root == ['Ave']:
        cells_to_be_used_as_root = set(c for loc in textual_mutation_dict for c in textual_mutation_dict[loc])
    else:
        cells_to_be_used_as_root = set(cells_to_be_used_as_root)
    root_collection = dict()
    for loc in textual_mutation_dict:
        for c in textual_mutation_dict[loc].keys() & cells_to_be_used_as_root:
            root_collection.setdefault(loc, []).append(textual_mutation_dict[loc][c])
    for loc in root_collection:
        val = int(np.median(root_collection[loc]))
        new_d.setdefault(loc, dict())['root'] = val
        new_d[loc].update(textual_mutation_dict[loc])
    return new_d


def create_FastTree_fasta_file(td, rldr, ind_name, title, min_group_size, rldr_group):
    td_with_root = add_root_to_dict(td, rldr)
    df = pd.DataFrame.from_dict(td_with_root)
    vs = list(v for vs in td_with_root.values() for v in vs.values())
    for i in range(min(vs), max(vs) + 4):
        if i not in symbol_map:
            symbol_map[i] = '-'

    df.replace(symbol_map, inplace=True)
    df.fillna("-", inplace=True)
    normalized_fasta_dir = fix_directories("/home/ofirr/s/trees", ind_name)
    normalized_fasta_path = '{}/{}_{}_FastTree_g{}_root_{}.fasta'.format(normalized_fasta_dir, title, ind_name,
                                                                   min_group_size, rldr_group)
    with open(normalized_fasta_path, 'w') as f:
        for sr, row in df.iterrows():
            f.write('>{}\r\n{}\r\n'.format(sr, ''.join(row.values)))

    return normalized_fasta_path


def run_fasttree_binary(newick_output_file, mutations_fasta_path, variation=fasttree_pseudo_trans_unified):
    variation('-out', newick_output_file, mutations_fasta_path)
    good_tree = dendropy.Tree.get_from_path(
        newick_output_file,
        "newick")
    return good_tree


def run_fastree(fasta_path, ind_name, title, min_group_size, rldr_group):
    newick_output_files = []
    normalized_newick_dir = fix_directories("/home/ofirr/s/trees", ind_name)
    for name, method, kwargs in [
        ('FTPU', 'fasttree_pseudo_trans_unified', {'variation': fasttree_pseudo_trans_unified}),
    ]:
        newick_output_file = '{}/{}_{}_g{}_{}_root_{}.newick'.format(normalized_newick_dir, title, ind_name,
                                                                     min_group_size, method, rldr_group)
        ret_tree = run_fasttree_binary(newick_output_file, fasta_path, list(kwargs.values())[0])
        newick_output_files.append(newick_output_file)

    return newick_output_files


def unroot_FastTree_tree(newick_output_file):
    rooted_tree = dendropy.Tree.get_from_path(newick_output_file, "newick")
    rooted_tree.reroot_at_node(rooted_tree.find_node_with_taxon_label('root'), update_bipartitions=False)
    tree = dendropy.Tree.get_from_string(rooted_tree.as_string('newick', suppress_rooting=False).replace('root', ''), 'newick')
    tree.ladderize(ascending=False)
    return tree


def fastree(td, rldr, ind_name, title, min_group_size, multi_run_srs, rldr_group):
    fasta_path = create_FastTree_fasta_file(td, rldr, ind_name, title, min_group_size, rldr_group)
    newick_output_files = run_fastree(fasta_path, ind_name, title, min_group_size, rldr_group)
    return newick_output_files

In [2]:
#Input files are dictionary input for fastree 'mutation_dict'
all_tdpath_by_ind_name = {
# #     'BC14-2680':  '/home/dcsoft/s/Ofir/td_rooted_doubles_test_all__updated4_BC14-2680.pickle',
# #     'BC15-0267':  '/home/dcsoft/s/Ofir/td_rooted_doubles_test_all__updated4_BC15-0267.pickle',
#     'BC15-0267':  '/home/ofirr/s/Ofir/td_2023_05_10_nodupmss_cleaned_BC15-0267.pickle',
# #     'BC15-0267':  "/home/dcsoft/s/Ofir/td_2023_04_20_nodupmss_cleaned_BC15-0267.pickle",
# #     'BC16-0401':  '/home/dcsoft/s/Ofir/td_rooted_doubles_test_all__updated4_BC16-0401.pickle',
# #     'BC16-0545':  '/home/dcsoft/s/Ofir/td_rooted_doubles_test_all__updated4_BC16-0545.pickle',
#     '775-13':  '/home/ofirr/s/Ofir/td_cleaned_all__updated4_775-13.pickle',
# #     '795-09':  '/home/dcsoft/s/Ofir/td_cleaned_all__updated4_795-09.pickle',
    '795-09':  '/home/ofirr/s/Ofir/td_cleaned_all__updated4_795-09.pickle',
# #     '795-09':  '/home/dcsoft/s/Ofir/td_cleaned_all__updated4_795-09_no8.pickle',
#     '841-12':  '/home/ofirr/s/Ofir/td_cleaned_all__updated4_841-12.pickle',
#     'MM15-127':'/home/ofirr/s/Ofir/td_cleaned_all__updated4_MM15-127.pickle',
# #     'MM15-127':'/home/dcsoft/s/Ofir/td_cleaned_doubles_test_all__updated4_MM15-127.pickle',
#     'MM16-423':'/home/ofirr/s/Ofir/td_cleaned_all__updated4_MM16-423.pickle',
#     'MM16-412':'/home/ofirr/s/Ofir/td_cleaned_all__updated4_MM16-412.pickle',
# #     'YUCLAT': '/home/dcsoft/s/Ofir/td_cleaned_all__updated5_YUCLAT.pickle'
# #     'YUCLAT_snps': '/home/dcsoft/s/trees/22_09_2020/YUCLAT/ftd_snps.pickle',
# #     'YUCLAT_strs': '/home/dcsoft/s/trees/22_09_2020/YUCLAT/td_cleaned_doubles_test_all_nomono_updated4_YUCLAT.pickle',
# #     'DU145-ex vivo experiment. Seeding started 12032014': '/home/dcsoft/s/Ofir/td_cleaned_all__updated4_DU145-ex vivo experiment. Seeding started 12032014.pickle',
}


In [3]:
import pickle
def ref_iteration(bsi, td_path, ind_name, rldr, srs, typ):
    with open(td_path, 'rb') as f:
        td = pickle.load(f)
#     bsi_td = {i:v for i, v in enumerate(random.choices(list(td.values()), k=len(td)))}
    bsi_td = td
    title = 'fasttree_yaara_{}_ref{}'.format(typ, bsi)
    rldr_group = 'Original'

    newick_output_files = fastree(bsi_td, rldr, ind_name, title, 0, srs, rldr_group)
    return newick_output_files

In [4]:
import pickle
def bootstrap_iteration(bsi, td_path, ind_name, rldr, srs, typ):
    with open(td_path, 'rb') as f:
        td = pickle.load(f)
    bsi_td = {i:v for i, v in enumerate(random.choices(list(td.values()), k=len(td)))}
#     bsi_td = td
    title = 'fasttree_yaara_{}_bsi{}'.format(typ, bsi)
    rldr_group = 'Original'

    newick_output_files = fastree(bsi_td, rldr, ind_name, title, 0, srs, rldr_group)
    return newick_output_files

In [5]:
labels_795_09 = {28823: 'CTC2016', 28825: 'CTC2016', 28844: 'CD3', 28879: 'CTC2017', 28895: 'CD3', 28903: 'CD3', 40216: 'PTsinglecells', 40217: 'PTsinglecells', 40218: 'PTsinglecells', 40219: 'PTsinglecells', 40220: 'PTsinglecells', 40221: 'PTsinglecells', 40222: 'PTsinglecells', 40223: 'PTsinglecells', 40224: 'PTsinglecells', 40226: 'PTsinglecells', 40227: 'PTsinglecells', 40228: 'PTsinglecells', 40229: 'PTsinglecells', 40230: 'PTsinglecells', 40232: 'PTsinglecells', 40233: 'PTsinglecells', 40234: 'PTsinglecells', 40235: 'PTsinglecells', 40236: 'PTsinglecells', 40238: 'PTsinglecells', 40239: 'DCC', 40240: 'DCC', 40241: 'CTC2016', 40242: 'CTC2016', 40243: 'CTC2016', 40244: 'CTC2016', 40245: 'CTC2016', 40246: 'CTC2016', 40247: 'CTC2016', 40248: 'CTC2016', 40249: 'CTC2016', 40250: 'CTC2016', 40251: 'CTC2017', 40252: 'CTC2017', 40253: 'CTC2017', 40254: 'CTC2017', 40255: 'CD68', 40256: 'CD68', 40257: 'CD68', 40258: 'CD68', 40259: 'CD68', 40260: 'CD68', 40261: 'CD68', 40262: 'CD68', 40263: 'CD68', 40264: 'CD68', 40265: 'CD68', 40266: 'CD68', 40267: 'CD68', 40268: 'CD68', 40269: 'CD68', 40270: 'CD68', 40271: 'CD68', 40272: 'CD68', 40273: 'CD68', 40274: 'CD68', 40275: 'CD68', 40276: 'CD68', 40277: 'CD3', 40278: 'CD3', 40279: 'CD3', 40280: 'CD3', 40281: 'CD3', 40282: 'CD3', 40283: 'CD3', 40284: 'CD3', 40285: 'CD3', 40286: 'CD3', 40287: 'CD3', 40288: 'CD3', 40289: 'CD3', 40290: 'CD3', 40291: 'CD3', 40292: 'CD3', 40293: 'CD3', 40294: 'CD3', 40295: 'CD3', 40296: 'CD3', 40297: 'CD3', 40300: 'OEC', 40301: 'OEC', 40302: 'OEC', 40303: 'OEC', 40304: 'OEC', 40305: 'OEC', 40306: 'OEC', 40308: 'OEC', 40310: 'Germline', 40312: 'Germline', 40314: 'Germline', 40315: 'Germline', 40316: 'Germline', 40317: 'Germline', 40318: 'Germline',}

In [6]:
BOOTSTRAP_ITER_NUM=1000
import random
i=1
l = []
for fil, pickles_dict in [
        ('all', all_tdpath_by_ind_name)
    ]:
    for ind_name in all_tdpath_by_ind_name:
        tdpath = pickles_dict[ind_name]
        with open(tdpath, 'rb') as f:
            td = pickle.load(f)
        d = transpose_dict(td)
        srs = [int(label[2:]) for label in d.keys() if label != 'root']
        rldr = ['ID{}'.format(srid) for srid in srs if labels_795_09[srid] == "Germline"]
#         l.append(executor.submit(bootstrap_iteration, i, td_path=tdpath, ind_name=ind_name, rldr=rldr, srs=srs,typ=fil))
        ref_iteration(i, td_path=tdpath, ind_name=ind_name, rldr=rldr, srs=srs,typ=fil)
        print(ind_name)

795-09


In [7]:
tree = dendropy.Tree.get(
    path="/home/ofirr/s/trees/05_06_2023/795-09/fasttree_yaara_all_ref1_795-09_g0_fasttree_pseudo_trans_unified_root_Original.newick", 
    schema="newick")

In [8]:
print(tree.as_ascii_plot(width=60))

                               /-------------------- ID40314
                               |                            
                          /----+              /----- ID40310
                          |    |    /---------+             
                          |    |    |         \----- ID40316
                          |    \----+                       
                          |         |    /---------- root   
/-------------------------+         \----+                  
|                         |              |    /----- ID40318
|                         |              \----+             
|                         |                   \----- ID40317
|                         |                                 
|                         |                   /----- ID40315
|                         \-------------------+             
|                                             \----- ID40312
|                                                           
|                       